We provide the following data to support experimentation with signal representation and classification techniques. Please acknowledge us in any communications of your work involving our data.

We have worked primarily with EEG data recorded by Zak Keirn at Purdue University for his work on his Masters of Science thesis in the Electrical Engineering Department at Purdue. We make that data available here as a 23 MB, binary Matlab mat-file. After downloading this file, load it into matlab using load eegdata. You should then have these two variables defined:

>> whos

Name     |   Size    |    Bytes | Class
---------|-----------|----------|-----------
data     |   1x325   | 22917020 | cell array
readme   |   1x1379  |     2758 | char array

Grand total is 5699264 elements using 22919778 bytes
The variable readme is a string containing the following explanation:

data is a cell array of cell arrays. Each individual cell array is made up of a subject string, task string, trial string, and data array. Each data array is 7 rows by 2500 columns. The 7 rows correspond to channels c3, c4, p3, p4, o1, o2, and EOG. Across columns are samples taken at 250 Hz for 10 seconds, for 2500 samples. For example, the first cell array looks like 'subject 1' 'baseline' 'trial 1' [7x2500 single]. Recordings were made with reference to electrically linked mastoids A1 and A2. EOG was recorded between the forehead above the left browline and another on the left cheekbone. Recording was performed with a bank of Grass 7P511 amplifiers whose bandpass analog filters were set at 0.1 to 100 Hz. Subjects 1 and 2 were employees of a university and were left-handed age 48 and right-handed age 39, respectively. Subjects 3 through 7 were right-handed college students between the age of 20 and 30 years old. All were mail subjects with the exception of Subject 5. Subjects performed five trials of each task in one day. They returned to do a second five trials on another day. Subjects 2 and 7 completed only one 5-trial session. Subject 5 completed three sessions. For more information see Alternative Modes of Communication Between Man and Machine, Zachary A. Keirn, Masters Thesis in Electrical Engineering, Purdue University, December, 1988.

Here is a file named makesubset.m that will extract five 7x2500 matrices for Subject 1, Trial 1 of each of the five tasks and plots them. This can be run only after loading the eegdata.mat file.

If you don't have access to Matlab, or want more data, here is a 12.8 MB file named alleegdata.ascii.gz that contains data for all subjects. Values are stored with a precision of three decimal places to save space.

Here is a tutorial in html showing how to access our EEG data stored locally and how to get started with visualizing and analyzing the data with matlab.